In [33]:
import pickle
import numpy as np
import pandas as pd

In [34]:
data_path = '../data/gene_list'
# load these files:
# tf_diff_all.pkl  tf_tf_all.pkl  tf_var_all.pkl  v2_tf_diff_all.pkl  v2_tf_tf_all.pkl  v2_tf_var_all.pkl
with open(data_path + '/tf_diff_all.pkl', 'rb') as f:
    tf_diff_all = pickle.load(f)

with open(data_path + '/tf_tf_all.pkl', 'rb') as f:
    tf_tf_all = pickle.load(f)

with open(data_path + '/tf_var_all.pkl', 'rb') as f:
    tf_var_all = pickle.load(f)

with open(data_path + '/v2_tf_diff_all.pkl', 'rb') as f:
    v2_tf_diff_all = pickle.load(f)

with open(data_path + '/v2_tf_tf_all.pkl', 'rb') as f:
    v2_tf_tf_all = pickle.load(f)

with open(data_path + '/v2_tf_var_all.pkl', 'rb') as f:
    v2_tf_var_all = pickle.load(f)

# '../data/alex_genes.txt', '../data/alex_tfs.txt'
with open('../data/alex_genes.txt', 'r') as f:
    alex_genes = f.read().splitlines()

with open('../data/alex_tfs.txt', 'r') as f:
    alex_tfs = f.read().splitlines()

with open('../data/hvg_50.txt', 'r') as f:
    hvg_50 = f.read().splitlines()

with open('../data/hv_tf_50.txt', 'r') as f:
    hv_tf_50 = f.read().splitlines()

with open('../data/hvg.txt', 'r') as f:
    hvg = f.read().splitlines()

with open('../data/hv_tf.txt', 'r') as f:
    hv_tf = f.read().splitlines()

with open('../data/all_genes.txt', 'r') as f:
    all_genes = f.read().splitlines()

with open('../data/all_tfs.txt', 'r') as f:
    all_tfs = f.read().splitlines()

draft_matrix = pd.read_excel('../data/top_regulated_genes_48_epi_tfs.xlsx', index_col=0)


In [35]:
print(f'tf_diff_all.shape: {tf_diff_all.shape}')
print(f'tf_tf_all.shape: {tf_tf_all.shape}')
print(f'tf_var_all.shape: {tf_var_all.shape}')
print(f'v2_tf_diff_all.shape: {v2_tf_diff_all.shape}')
print(f'v2_tf_tf_all.shape: {v2_tf_tf_all.shape}')
print(f'v2_tf_var_all.shape: {v2_tf_var_all.shape}')
print(f'draft_matrix.shape: {draft_matrix.shape}')
print(f'alex_genes: {len(alex_genes)}')
print(f'alex_tfs: {len(alex_tfs)}')
print(f'hvg_50: {len(hvg_50)}')
print(f'hv_tf_50: {len(hv_tf_50)}')
print(f'hvg: {len(hvg)}')
print(f'hv_tf: {len(hv_tf)}')
print(f'all_genes: {len(all_genes)}')
print(f'all_tfs: {len(all_tfs)}')


tf_diff_all.shape: (330, 1008)
tf_tf_all.shape: (330, 330)
tf_var_all.shape: (330, 1968)
v2_tf_diff_all.shape: (330, 1008)
v2_tf_tf_all.shape: (330, 330)
v2_tf_var_all.shape: (330, 1968)
draft_matrix.shape: (276, 48)
alex_genes: 1970
alex_tfs: 331
hvg_50: 10701
hv_tf_50: 640
hvg: 5351
hv_tf: 320
all_genes: 21465
all_tfs: 1345


In [36]:
# Get indices from each dataframe and strip _x suffix for rows
tf_diff_indices = {i.split('_')[0] for i in tf_diff_all.index}
tf_tf_indices = {i.split('_')[0] for i in tf_tf_all.index}
tf_var_indices = {i.split('_')[0] for i in tf_var_all.index}
v2_tf_diff_indices = {i.split('_')[0] for i in v2_tf_diff_all.index}
v2_tf_tf_indices = {i.split('_')[0] for i in v2_tf_tf_all.index}
v2_tf_var_indices = {i.split('_')[0] for i in v2_tf_var_all.index}
alex_tf_indices = set(alex_tfs)
hv_tf_indices = set(hv_tf)
hv_tf_50_indices = set(hv_tf_50)
draft_indices = {i.split('_')[0] for i in draft_matrix.T.index}
all_tf_indices = set(all_tfs)

# Create list of all row index sets
row_indices = [tf_diff_indices, tf_tf_indices, tf_var_indices,
               v2_tf_diff_indices, v2_tf_tf_indices, v2_tf_var_indices, 
               alex_tf_indices, hv_tf_indices, hv_tf_50_indices, draft_indices, all_tf_indices]

# Get column indices from relevant dataframes and strip _y suffix
tf_diff_cols = {c.split('_')[0] for c in tf_diff_all.columns}
tf_var_cols = {c.split('_')[0] for c in tf_var_all.columns}
v2_tf_diff_cols = {c.split('_')[0] for c in v2_tf_diff_all.columns}
v2_tf_var_cols = {c.split('_')[0] for c in v2_tf_var_all.columns}
alex_gene_cols = set(alex_genes)
hvg_cols = set(hvg)
hvg_50_cols = set(hvg_50)
draft_cols = {c.split('_')[0] for c in draft_matrix.T.columns}
all_gene_cols = set(all_genes)

# Create list of all column index sets (only non-tf_tf dataframes and alex_genes)
col_indices = [tf_diff_cols, tf_var_cols, v2_tf_diff_cols, v2_tf_var_cols, 
               alex_gene_cols, hvg_cols, hvg_50_cols, draft_cols, all_gene_cols]

# Create labels for the matrices
row_labels = ['tf_diff', 'tf_tf', 'tf_var', 'v2_tf_diff', 'v2_tf_tf', 'v2_tf_var', 
              'alex_tfs', 'hv_tf', 'hv_tf_50', 'draft', 'all_tfs']
col_labels = ['tf_diff', 'tf_var', 'v2_tf_diff', 'v2_tf_var', 
              'alex_genes', 'hvg', 'hvg_50', 'draft', 'all_genes']

# Create empty matrices
row_matrix = np.zeros((len(row_indices), len(row_indices)))
col_matrix = np.zeros((len(col_indices), len(col_indices)))

# Fill row matrix with intersection lengths
for i in range(len(row_indices)):
    for j in range(len(row_indices)):
        row_matrix[i,j] = len(row_indices[i].intersection(row_indices[j]))

# Fill column matrix with intersection lengths
for i in range(len(col_indices)):
    for j in range(len(col_indices)):
        col_matrix[i,j] = len(col_indices[i].intersection(col_indices[j]))

# Print matrices with labels
print("Row intersection sizes:")
print("            " + "  ".join(f"{label:>10}" for label in row_labels))
for i, label in enumerate(row_labels):
    print(f"{label:10} " + "  ".join(f"{int(size):10d}" for size in row_matrix[i]))

print("\nColumn intersection sizes:")
print("            " + "  ".join(f"{label:>10}" for label in col_labels))
for i, label in enumerate(col_labels):
    print(f"{label:10} " + "  ".join(f"{int(size):10d}" for size in col_matrix[i]))


Row intersection sizes:
               tf_diff       tf_tf      tf_var  v2_tf_diff    v2_tf_tf   v2_tf_var    alex_tfs       hv_tf    hv_tf_50       draft     all_tfs
tf_diff           330         330         330         330         330         330         330         144         230          32         330
tf_tf             330         330         330         330         330         330         330         144         230          32         330
tf_var            330         330         330         330         330         330         330         144         230          32         330
v2_tf_diff        330         330         330         330         330         330         330         144         230          32         330
v2_tf_tf          330         330         330         330         330         330         330         144         230          32         330
v2_tf_var         330         330         330         330         330         330         330         144         230      

In [37]:
len(alex_genes)

1970

In [38]:
len(alex_tfs)

331

In [39]:
v2_tf_diff_all

,SAMD11_y,NOC2L_y,HES4_y,ISG15_y,MAD2L2_y,PDPN_y,KAZN_y,AL450998.2_y,EPHA2_y,ARHGEF19_y,...,MT-CO1_y,MT-CO2_y,MT-ATP8_y,MT-ATP6_y,MT-CO3_y,MT-ND3_y,MT-ND4_y,MT-ND5_y,MT-ND6_y,MT-CYB_y
HES4_x,4.733087e-08,7.573487e-13,1.000000e+00,7.033291e-02,5.144467e-13,0.058187,7.321958e-87,8.004790e-01,0.002546,1.065304e-03,...,0.916261,1.946203e-39,0.000066,2.947148e-02,0.129364,5.696517e-05,3.960828e-02,8.816229e-29,6.673081e-09,1.185529e-01
CENPS_x,1.201276e-02,3.789806e-04,4.696935e-01,8.455412e-01,9.686715e-01,0.725200,3.590946e-01,9.776650e-01,0.834556,2.168013e-03,...,0.334941,4.371708e-02,0.713005,8.348776e-02,0.000999,5.560299e-01,7.407693e-16,3.444254e-01,6.843124e-18,3.365171e-02
RUNX3_x,3.063082e-03,3.053760e-04,7.808908e-03,1.894875e-11,2.513334e-05,0.481698,3.806253e-01,2.142634e-05,0.000110,1.066584e-01,...,0.211003,1.361203e-03,0.235038,7.618685e-01,0.257045,1.260186e-01,5.696311e-04,4.960791e-18,2.078263e-03,9.729811e-01
AHDC1_x,2.363101e-01,2.558037e-01,2.179046e-01,3.837119e-03,1.887589e-02,0.039331,1.689736e-10,3.314119e-01,0.001687,4.448243e-01,...,0.307918,3.544899e-46,0.002731,4.620024e-01,0.693979,1.578794e-02,8.500388e-31,1.501404e-04,2.987295e-01,9.126500e-01
ZBTB8A_x,2.785631e-04,2.662019e-04,9.478420e-01,2.328556e-03,8.482203e-08,0.032315,9.114423e-10,9.068896e-01,0.002066,1.249610e-02,...,0.700598,1.579786e-27,0.000011,5.970527e-02,0.399660,5.119239e-05,1.254129e-03,7.485172e-24,6.691953e-04,2.235277e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GABPA_x,6.538444e-02,2.266019e-01,9.821793e-01,2.308007e-01,1.351946e-04,0.089954,9.499388e-01,4.835027e-01,0.443396,1.203416e-05,...,0.752420,4.182140e-01,0.006299,1.080105e-34,0.015348,9.064316e-07,1.213591e-05,2.946826e-01,9.335949e-02,2.097571e-15
BACH1_x,8.494415e-01,6.825631e-01,1.581463e-01,3.788367e-01,4.288825e-01,0.824893,5.809493e-03,4.821832e-01,0.091025,2.320968e-01,...,0.013778,7.374415e-26,0.002481,9.631432e-12,0.001108,2.035389e-09,1.534803e-04,1.661886e-03,6.414196e-01,1.652724e-10
SON_x,9.539976e-82,3.091160e-39,1.400204e-18,2.054963e-03,5.641104e-57,0.528967,9.627549e-04,6.884449e-14,0.008772,5.129530e-11,...,0.215177,1.958673e-01,0.000519,2.381265e-02,0.005233,8.447869e-09,2.917688e-01,3.393067e-13,2.864229e-68,1.315030e-01
RUNX1_x,9.668414e-01,2.801393e-01,9.938560e-01,4.793888e-01,6.208877e-01,0.628246,6.037357e-01,4.680527e-01,0.781258,2.312194e-01,...,0.713792,1.175196e-01,0.717424,1.788019e-01,0.009630,6.897251e-01,5.525512e-14,3.658781e-01,8.749391e-01,7.879618e-02
